In [1]:
# In this version of OGA with random dictionaries, we use QMC to evaluate the loss function. 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_dtype(torch.float64)


class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    

def show_convergence_order(err_l2,exponent,dict_size, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t {:.6f} \t\t *  \n".format(neuron_nums[i],item ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t \n".format(neuron_nums[i],item ))
        else: 
            print("{} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()

def show_convergence_order_latex(err_l2,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    l2_order = -1/2-(2*k + 1)/(2*d)
    print("neuron num  & \t $\\|u-u_n \\|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  \\\\ \\hline \\hline ".format(l2_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t & {:.6f} &\t\t *  \\\ \hline  \n".format(neuron_nums[i],item) )   
        else: 
            print("{} \t\t &  {:.3e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )


def print_convergence_order(err, neuron_num_exponent): 

    neuron_nums = [2**j for j in range(2,neuron_num_exponent)]
    err_list = [err[i] for i in neuron_nums ] 

    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))



def generate_relu_dict4D(N_list):
    
    N = np.prod(N_list) 

    grid_indices = [np.linspace(0,1,N_item,endpoint=False) for N_item in N_list]
    grid = np.meshgrid(*grid_indices,indexing='ij')
    grid_coordinates = np.column_stack([axis.ravel() for axis in grid]) 
    samples = torch.tensor(grid_coordinates) 

    T =torch.tensor([[pi,0,0,0],[0,pi,0,0],[0,0,2*pi,0],[0,0,0,2*2]]) # 2 * sqrt(d)
    shift = torch.tensor([0,0,0,-2])
    samples = samples@T + shift 

    f1 = torch.zeros(N,1) 
    f2 = torch.zeros(N,1)
    f3 = torch.zeros(N,1)
    f4 = torch.zeros(N,1)
    f5 = torch.zeros(N,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.cos(samples[:,2])
    f4[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.sin(samples[:,2])  
    f5[:,0] = samples[:,3]

    Wb_tensor = torch.cat([f1,f2,f3,f4,f5],1) # N x 4 
    return Wb_tensor.to(device)


def generate_relu_dict4D_QMC(s,N0):
    # Sob = torch.quasirandom.SobolEngine(dimension =4, scramble= True, seed=None) 
    # samples = Sob.draw(N0).double() 

    # for i in range(s-1):
    #     samples = torch.cat([samples,Sob.draw(N0).double()],0)

    # Monte Carlo 
    samples = torch.rand(s*N0,4) 

    T =torch.tensor([[pi,0,0,0],[0,pi,0,0],[0,0,2*pi,0],[0,0,0,2*2]])
    shift = torch.tensor([0,0,0,-2])
    samples = samples@T + shift 

    f1 = torch.zeros(s*N0,1) 
    f2 = torch.zeros(s*N0,1)
    f3 = torch.zeros(s*N0,1)
    f4 = torch.zeros(s*N0,1)
    f5 = torch.zeros(s*N0,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.cos(samples[:,2])
    f4[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.sin(samples[:,2])  
    f5[:,0] = samples[:,3]

    Wb_tensor = torch.cat([f1,f2,f3,f4,f5],1) # N x 4 
    return Wb_tensor.to(device)

def adjust_neuron_position(my_model, dims = 3):

    def create_mesh_grid(dims, pts):
        mesh = torch.tensor(list(itertools.product(pts,repeat=dims)))
        vertices = mesh.reshape(len(pts) ** dims, -1) 
        return vertices
    counter = 0 
    # positions = torch.tensor([[0.,0.],[0.,1.],[1.,1.],[1.,0.]])
    pts = torch.tensor([0.,1.])
    positions = create_mesh_grid(dims,pts) 
    neuron_num = my_model.fc1.bias.size(0)
    for i in range(neuron_num): 
        w = my_model.fc1.weight.data[i:i+1,:]
        b = my_model.fc1.bias.data[i]
    #     print(w,b)
        values = torch.matmul(positions,w.T) # + b
        left_end = - torch.max(values)
        right_end = - torch.min(values)
        offset = (right_end - left_end)/50
        if b <= left_end + offset/2 : 
            b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
            my_model.fc1.bias.data[i] = b 
        if b >= right_end - offset/2 :
            if counter < (dims+1):
#                 print("here")
                counter += 1
            else: # (d + 1) or more 
                b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
                my_model.fc1.bias.data[i] = b 
    return my_model

def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights.to(device), integration_points.to(device) 


def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct"):
    """assemble the linear system and solve it 
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    jac = weighted_basis_value_col.t() @ basis_value_col 
     
    rhs = weighted_basis_value_col.t() @ (target(integration_points)) 
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 

def OGAL2FittingReLU4D(my_model,target,N_list,num_epochs, M, k =1, linear_solver = "direct",num_batches = 1): 
    
    """ Orthogonal greedy algorithm (0,1)^4 
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 
    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """

    weights, integration_points = MonteCarlo_Sobol_dDim_weights_points(M ,d = 4) 

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values

    err[0]= torch.mean(func_values_sqrd)**0.5
    all_start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    N = np.prod(N_list) 
    relu_dict_parameters = generate_relu_dict4D(N_list).t() 
#     num_batches = 2**3 #8, divide dictionary elements 
    batch_size = N//num_batches # batch_size * i : batch_size * (i+1), i = 0,..., num_batches - 1 
    
    for i in range(num_epochs): 
  
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            with torch.no_grad(): 
                func_values = target(integration_points) - my_model(integration_points)

        start_time = time.time() 

        output = torch.zeros(N,1)
        
        for j in range(0,N,batch_size): 

            end_index = j + batch_size  
            basis_values_batch = (F.relu( torch.matmul(integration_points,relu_dict_parameters[0:4, j:end_index] ) - relu_dict_parameters[4, j:end_index])**k).T # uses broadcasting    
            output[j:end_index,0]  = (torch.abs(torch.matmul(basis_values_batch,func_values))/M)[:,0]
            
        neuron_index = torch.argmax(output.flatten())
        
        print("argmax time taken, ", time.time() - start_time)
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[0:4, neuron_index]) # 
        list_b.append(-relu_dict_parameters[4,neuron_index])
        num_neuron += 1
        my_model = model(4,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        #Todo 
        start_time = time.time() 
        sol = minimize_linear_layer_explicit_assemble(my_model,target,weights,integration_points, solver)
        print("\t\t time taken minimize linear layer: ",time.time() - start_time) 
        my_model.fc2.weight.data[0,:] = sol[:]

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.mean(func_values_sqrd)**0.5
        print("current error: ",err[i+1]) 
    print("time taken: ",time.time() - all_start_time)
    return err, my_model


def OGAL2FittingReLU4D_QMC(my_model,target,s,N0,num_epochs, M, k =1, linear_solver = "direct", num_batches = 1): 
    
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    # samples for QMC integral
    start_time = time.time()
    # Sob_integral = torch.quasirandom.SobolEngine(dimension =4, scramble= False, seed=None) 
    # integration_points = Sob_integral.draw(M).double() 
    # integration_points = integration_points.to(device)
    weights, integration_points = MonteCarlo_Sobol_dDim_weights_points(M ,d = 4) 
    print("generate sob sequence:", time.time() - start_time) 

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.mean(func_values_sqrd)**0.5
    all_start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict4D_QMC(s,N0).t()   
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        start_time = time.time() 
        
        N = s*N0 
        output = torch.zeros(N,1)
        batch_size = N//num_batches
        
        for j in range(0,N,batch_size): 

            end_index = j + batch_size  
            basis_values_batch = (F.relu( torch.matmul(integration_points,relu_dict_parameters[0:4, j:end_index] ) - relu_dict_parameters[4, j:end_index])**k).T # uses broadcasting    
            output[j:end_index,0]  = (torch.abs(torch.matmul(basis_values_batch,func_values))/M)[:,0]
            
        neuron_index = torch.argmax(output.flatten())
        
#         basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T # uses broadcasting
#         output = torch.abs(torch.matmul(basis_values,func_values))/M # 
#         neuron_index = torch.argmax(output.flatten())
        print("argmax time taken, ", time.time() - start_time)
        
        # print(neuron_index)
#         list_w.append(relu_dict_parameters[neuron_index,0:4]) # 
#         list_b.append(-relu_dict_parameters[neuron_index,4])
        list_w.append(relu_dict_parameters[0:4, neuron_index]) # 
        list_b.append(-relu_dict_parameters[4,neuron_index])
        num_neuron += 1
        my_model = model(4,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        #Todo 
        start_time = time.time() 
        sol = minimize_linear_layer_explicit_assemble(my_model,target,weights,integration_points, solver)
        print("\t\t time taken minimize linear layer: ",time.time() - start_time) 
        my_model.fc2.weight.data[0,:] = sol[:]

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.mean(func_values_sqrd)**0.5
        print("current error: ",err[i+1]) 
    print("total duration: ",time.time() - all_start_time)
    return err, my_model


## Deterministic dictionary 

In [4]:

def target(x):
    return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) *torch.sin(pi*x[:,3:4])

dim = 4 
function_name = "sin-product-4d" 
filename_write = "data/2DOGA-{}-order.txt".format(function_name)
M = 2**19 # MC points around 50w 
f_write = open(filename_write, "a")
f_write.write("Integration points: Quasi Monte Carlo:  {}\n".format(M))
f_write.close() 
save = True 
write2file = True

for relu_k in [4]: 
    for N_list in [[2**4, 2**4, 2**3, 2**3]]: 
        N = np.prod(N_list) 
        print()
        print() 
        exponent = 9  
        num_epochs=  2**exponent 
    
        my_model = None 
        err, my_model = OGAL2FittingReLU4D(my_model,target,N_list,num_epochs, M, k = relu_k, linear_solver = "ls", num_batches= 2**3)
        
        if save: 
            folder = 'data/'
            filename = folder + function_name + "_err_deterministic_Dict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,N,num_epochs)
            torch.save(err,filename)
            filename = folder + function_name + "_model_deterministic_Dict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,N,num_epochs)
            torch.save(my_model.state_dict(),filename) 
            
        show_convergence_order(err,exponent,N,filename_write,write2file = write2file)
        show_convergence_order_latex(err,exponent,k=relu_k,d=dim)






using linear solver:  ls
epoch:  1	argmax time taken,  0.43138813972473145
assembling the matrix time taken:  0.0002498626708984375
solving Ax = b time taken:  0.00015544891357421875
		 time taken minimize linear layer:  0.0004353523254394531
current error:  tensor(0.2000)
epoch:  2	argmax time taken,  0.41163158416748047
assembling the matrix time taken:  0.0002880096435546875
solving Ax = b time taken:  0.00026154518127441406
		 time taken minimize linear layer:  0.0005805492401123047
current error:  tensor(0.1988)
epoch:  3	argmax time taken,  0.4120492935180664
assembling the matrix time taken:  0.0002505779266357422
solving Ax = b time taken:  0.00029468536376953125
		 time taken minimize linear layer:  0.0005764961242675781
current error:  tensor(0.1954)
epoch:  4	argmax time taken,  0.41201114654541016
assembling the matrix time taken:  0.0002665519714355469
solving Ax = b time taken:  0.0003178119659423828
		 time taken minimize linear layer:  0.0006160736083984375
current er

epoch:  34	argmax time taken,  0.41401195526123047
assembling the matrix time taken:  0.00027680397033691406
solving Ax = b time taken:  0.0015137195587158203
		 time taken minimize linear layer:  0.0018229484558105469
current error:  tensor(0.0972)
epoch:  35	argmax time taken,  0.4142005443572998
assembling the matrix time taken:  0.0002543926239013672
solving Ax = b time taken:  0.001592397689819336
		 time taken minimize linear layer:  0.0018787384033203125
current error:  tensor(0.0925)
epoch:  36	argmax time taken,  0.4140441417694092
assembling the matrix time taken:  0.00026226043701171875
solving Ax = b time taken:  0.0015718936920166016
		 time taken minimize linear layer:  0.0018663406372070312
current error:  tensor(0.0885)
epoch:  37	argmax time taken,  0.4140913486480713
assembling the matrix time taken:  0.0002529621124267578
solving Ax = b time taken:  0.001638650894165039
		 time taken minimize linear layer:  0.0019235610961914062
current error:  tensor(0.0849)
epoch: 

epoch:  68	argmax time taken,  0.4148569107055664
assembling the matrix time taken:  0.0002636909484863281
solving Ax = b time taken:  0.003436565399169922
		 time taken minimize linear layer:  0.0037326812744140625
current error:  tensor(0.0377)
epoch:  69	argmax time taken,  0.41465282440185547
assembling the matrix time taken:  0.0002589225769042969
solving Ax = b time taken:  0.0035605430603027344
		 time taken minimize linear layer:  0.003851652145385742
current error:  tensor(0.0374)
epoch:  70	argmax time taken,  0.4147167205810547
assembling the matrix time taken:  0.00026226043701171875
solving Ax = b time taken:  0.003690958023071289
		 time taken minimize linear layer:  0.003985404968261719
current error:  tensor(0.0370)
epoch:  71	argmax time taken,  0.41517210006713867
assembling the matrix time taken:  0.0002498626708984375
solving Ax = b time taken:  0.0036847591400146484
		 time taken minimize linear layer:  0.003967761993408203
current error:  tensor(0.0369)
epoch:  72

epoch:  102	argmax time taken,  0.41730237007141113
assembling the matrix time taken:  0.00026488304138183594
solving Ax = b time taken:  0.005480766296386719
		 time taken minimize linear layer:  0.0057790279388427734
current error:  tensor(0.0167)
epoch:  103	argmax time taken,  0.4156060218811035
assembling the matrix time taken:  0.0002613067626953125
solving Ax = b time taken:  0.005346775054931641
		 time taken minimize linear layer:  0.005640745162963867
current error:  tensor(0.0165)
epoch:  104	argmax time taken,  0.4154651165008545
assembling the matrix time taken:  0.00026345252990722656
solving Ax = b time taken:  0.005247592926025391
		 time taken minimize linear layer:  0.005543708801269531
current error:  tensor(0.0163)
epoch:  105	argmax time taken,  0.41558170318603516
assembling the matrix time taken:  0.0002510547637939453
solving Ax = b time taken:  0.005369901657104492
		 time taken minimize linear layer:  0.0056536197662353516
current error:  tensor(0.0161)
epoch:

epoch:  135	argmax time taken,  0.41641688346862793
assembling the matrix time taken:  0.00025582313537597656
solving Ax = b time taken:  0.00821685791015625
		 time taken minimize linear layer:  0.008506536483764648
current error:  tensor(0.0105)
epoch:  136	argmax time taken,  0.4164714813232422
assembling the matrix time taken:  0.00026297569274902344
solving Ax = b time taken:  0.008295774459838867
		 time taken minimize linear layer:  0.008591175079345703
current error:  tensor(0.0105)
epoch:  137	argmax time taken,  0.416400671005249
assembling the matrix time taken:  0.0002484321594238281
solving Ax = b time taken:  0.008372068405151367
		 time taken minimize linear layer:  0.008653879165649414
current error:  tensor(0.0103)
epoch:  138	argmax time taken,  0.4167788028717041
assembling the matrix time taken:  0.0002589225769042969
solving Ax = b time taken:  0.008634805679321289
		 time taken minimize linear layer:  0.008926153182983398
current error:  tensor(0.0103)
epoch:  139

...

epoch:  373	argmax time taken,  0.42214107513427734
assembling the matrix time taken:  0.000247955322265625
solving Ax = b time taken:  0.03377079963684082
		 time taken minimize linear layer:  0.034050941467285156
current error:  tensor(0.0050)
epoch:  374	argmax time taken,  0.42214345932006836
assembling the matrix time taken:  0.0002536773681640625
solving Ax = b time taken:  0.03343605995178223
		 time taken minimize linear layer:  0.0337224006652832
current error:  tensor(0.0050)
epoch:  375	argmax time taken,  0.42226696014404297
assembling the matrix time taken:  0.00025391578674316406
solving Ax = b time taken:  0.033879995346069336
		 time taken minimize linear layer:  0.03416752815246582
current error:  tensor(0.0049)
epoch:  376	argmax time taken,  0.42233967781066895
assembling the matrix time taken:  0.0002582073211669922
solving Ax = b time taken:  0.03339505195617676
		 time taken minimize linear layer:  0.033685922622680664
current error:  tensor(0.0050)
epoch:  377	ar

epoch:  407	argmax time taken,  0.4227898120880127
assembling the matrix time taken:  0.0002491474151611328
solving Ax = b time taken:  0.03841757774353027
		 time taken minimize linear layer:  0.038701772689819336
current error:  tensor(0.0053)
epoch:  408	argmax time taken,  0.42273950576782227
assembling the matrix time taken:  0.00025582313537597656
solving Ax = b time taken:  0.03903770446777344
		 time taken minimize linear layer:  0.03932666778564453
current error:  tensor(0.0051)
epoch:  409	argmax time taken,  0.4228515625
assembling the matrix time taken:  0.0002465248107910156
solving Ax = b time taken:  0.03870964050292969
		 time taken minimize linear layer:  0.03898882865905762
current error:  tensor(0.0053)
epoch:  410	argmax time taken,  0.4228842258453369
assembling the matrix time taken:  0.00025081634521484375
solving Ax = b time taken:  0.03845334053039551
		 time taken minimize linear layer:  0.03873729705810547
current error:  tensor(0.0050)
epoch:  411	argmax tim

epoch:  441	argmax time taken,  0.42343664169311523
assembling the matrix time taken:  0.0002465248107910156
solving Ax = b time taken:  0.042064666748046875
		 time taken minimize linear layer:  0.04234504699707031
current error:  tensor(0.0054)
epoch:  442	argmax time taken,  0.4235506057739258
assembling the matrix time taken:  0.00025010108947753906
solving Ax = b time taken:  0.04268074035644531
		 time taken minimize linear layer:  0.04296422004699707
current error:  tensor(0.0047)
epoch:  443	argmax time taken,  0.4234731197357178
assembling the matrix time taken:  0.00025177001953125
solving Ax = b time taken:  0.04217791557312012
		 time taken minimize linear layer:  0.042463064193725586
current error:  tensor(0.0047)
epoch:  444	argmax time taken,  0.4235708713531494
assembling the matrix time taken:  0.0002560615539550781
solving Ax = b time taken:  0.04253435134887695
		 time taken minimize linear layer:  0.04282331466674805
current error:  tensor(0.0048)
epoch:  445	argmax

epoch:  475	argmax time taken,  0.4276094436645508
assembling the matrix time taken:  0.0002498626708984375
solving Ax = b time taken:  0.049265384674072266
		 time taken minimize linear layer:  0.04954886436462402
current error:  tensor(0.0048)
epoch:  476	argmax time taken,  0.42748188972473145
assembling the matrix time taken:  0.00026035308837890625
solving Ax = b time taken:  0.04866170883178711
		 time taken minimize linear layer:  0.04895496368408203
current error:  tensor(0.0049)
epoch:  477	argmax time taken,  0.4258441925048828
assembling the matrix time taken:  0.00024890899658203125
solving Ax = b time taken:  0.04941272735595703
		 time taken minimize linear layer:  0.04969477653503418
current error:  tensor(0.0053)
epoch:  478	argmax time taken,  0.4274611473083496
assembling the matrix time taken:  0.0002598762512207031
solving Ax = b time taken:  0.04929208755493164
		 time taken minimize linear layer:  0.04958486557006836
current error:  tensor(0.0048)
epoch:  479	argm

epoch:  509	argmax time taken,  0.4246494770050049
assembling the matrix time taken:  0.00024819374084472656
solving Ax = b time taken:  0.05387544631958008
		 time taken minimize linear layer:  0.05415606498718262
current error:  tensor(0.0047)
epoch:  510	argmax time taken,  0.4280703067779541
assembling the matrix time taken:  0.000247955322265625
solving Ax = b time taken:  0.05375933647155762
		 time taken minimize linear layer:  0.054039955139160156
current error:  tensor(0.0055)
epoch:  511	argmax time taken,  0.4248237609863281
assembling the matrix time taken:  0.00024628639221191406
solving Ax = b time taken:  0.05480837821960449
		 time taken minimize linear layer:  0.05508732795715332
current error:  tensor(0.0047)
epoch:  512	argmax time taken,  0.42475390434265137
assembling the matrix time taken:  0.00025010108947753906
solving Ax = b time taken:  0.06059622764587402
		 time taken minimize linear layer:  0.06087994575500488
current error:  tensor(0.0048)
time taken:  232

## Random dictionary 

In [6]:


def target(x):
    return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) *torch.sin(pi*x[:,3:4])

dim = 4 
function_name = "sin-product-4d" 
filename_write = "data/2DQMCOGA-{}-order.txt".format(function_name)
M = 2**19 # MC points around 50w 
f_write = open(filename_write, "a")
f_write.write("Integration points: Quasi Monte Carlo:  {}\n".format(M))
f_write.close() 
save = True 
write2file = True

for relu_k in [1]: 
    s = 1 
    for N0 in [2**6, 2**7, 2**8]: 
        N = np.prod(N_list) 
        print()
        print() 
        exponent = 9  
        num_epochs=  2**exponent 
    
        my_model = None 
        err, my_model = OGAL2FittingReLU4D_QMC(my_model,target, \
                    s,N0,num_epochs, M, k = relu_k, linear_solver = "direct", num_batches = 1)

        if save: 
            folder = 'data/'
            filename = folder + function_name + "_err_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(err,filename)
            filename = folder + function_name + "_model_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(my_model.state_dict(),filename) 
            
        show_convergence_order(err,exponent,N,filename_write,write2file = write2file)
        show_convergence_order_latex(err,exponent,k=relu_k,d=dim)




generate sob sequence: 0.005876302719116211
using linear solver:  direct
epoch:  1	argmax time taken,  0.0014488697052001953
assembling the matrix time taken:  0.00025653839111328125
solving Ax = b time taken:  3.698288679122925
		 time taken minimize linear layer:  3.698791742324829
current error:  tensor(0.1894)
epoch:  2	argmax time taken,  0.001430511474609375
assembling the matrix time taken:  0.0002942085266113281
solving Ax = b time taken:  0.0002727508544921875
		 time taken minimize linear layer:  0.0005979537963867188
current error:  tensor(0.1856)
epoch:  3	argmax time taken,  0.0015816688537597656
assembling the matrix time taken:  0.00024771690368652344
solving Ax = b time taken:  0.0003185272216796875
		 time taken minimize linear layer:  0.0005962848663330078
current error:  tensor(0.1795)
epoch:  4	argmax time taken,  0.0015726089477539062
assembling the matrix time taken:  0.0002644062042236328
solving Ax = b time taken:  0.0003197193145751953
		 time taken minimize 

assembling the matrix time taken:  0.0006504058837890625
solving Ax = b time taken:  0.001077413558959961
		 time taken minimize linear layer:  0.0017828941345214844
current error:  tensor(0.0630)
epoch:  39	argmax time taken,  0.004014730453491211
assembling the matrix time taken:  0.0002467632293701172
solving Ax = b time taken:  0.00154876708984375
		 time taken minimize linear layer:  0.0018262863159179688
current error:  tensor(0.0621)
epoch:  40	argmax time taken,  0.0040130615234375
assembling the matrix time taken:  0.0002582073211669922
solving Ax = b time taken:  0.0015077590942382812
		 time taken minimize linear layer:  0.001796722412109375
current error:  tensor(0.0615)
epoch:  41	argmax time taken,  0.004094600677490234
assembling the matrix time taken:  0.00025725364685058594
solving Ax = b time taken:  0.0014429092407226562
		 time taken minimize linear layer:  0.0017309188842773438
current error:  tensor(0.0608)
epoch:  42	argmax time taken,  0.0036172866821289062
asse

epoch:  76	argmax time taken,  0.004743814468383789
assembling the matrix time taken:  0.00026726722717285156
solving Ax = b time taken:  0.003073453903198242
		 time taken minimize linear layer:  0.0033712387084960938
current error:  tensor(0.0394)
epoch:  77	argmax time taken,  0.004072666168212891
assembling the matrix time taken:  0.00024700164794921875
solving Ax = b time taken:  0.003312349319458008
		 time taken minimize linear layer:  0.0035898685455322266
current error:  tensor(0.0391)
epoch:  78	argmax time taken,  0.00501251220703125
assembling the matrix time taken:  0.0002701282501220703
solving Ax = b time taken:  0.0034525394439697266
		 time taken minimize linear layer:  0.0037529468536376953
current error:  tensor(0.0388)
epoch:  79	argmax time taken,  0.004117250442504883
assembling the matrix time taken:  0.00024700164794921875
solving Ax = b time taken:  0.0033762454986572266
		 time taken minimize linear layer:  0.0036542415618896484
current error:  tensor(0.0386)


epoch:  114	argmax time taken,  0.004586696624755859
assembling the matrix time taken:  0.00025272369384765625
solving Ax = b time taken:  0.0042078495025634766
		 time taken minimize linear layer:  0.004491567611694336
current error:  tensor(0.0299)
epoch:  115	argmax time taken,  0.0063059329986572266
assembling the matrix time taken:  0.00025177001953125
solving Ax = b time taken:  0.0042514801025390625
		 time taken minimize linear layer:  0.004533529281616211
current error:  tensor(0.0296)
epoch:  116	argmax time taken,  0.0046045780181884766
assembling the matrix time taken:  0.00025200843811035156
solving Ax = b time taken:  0.0042324066162109375
		 time taken minimize linear layer:  0.004515171051025391
current error:  tensor(0.0295)
epoch:  117	argmax time taken,  0.004606008529663086
assembling the matrix time taken:  0.00025391578674316406
solving Ax = b time taken:  0.004292964935302734
		 time taken minimize linear layer:  0.0045778751373291016
current error:  tensor(0.029

current error:  tensor(0.0241)
epoch:  147	argmax time taken,  0.006034374237060547
assembling the matrix time taken:  0.0002532005310058594
solving Ax = b time taken:  0.007651567459106445
		 time taken minimize linear layer:  0.007935523986816406
current error:  tensor(0.0239)
epoch:  148	argmax time taken,  0.006742238998413086
assembling the matrix time taken:  0.00025725364685058594
solving Ax = b time taken:  0.0073888301849365234
		 time taken minimize linear layer:  0.007677316665649414
current error:  tensor(0.0237)
epoch:  149	argmax time taken,  0.006804227828979492
assembling the matrix time taken:  0.00026535987854003906
solving Ax = b time taken:  0.0075190067291259766
		 time taken minimize linear layer:  0.007815361022949219
current error:  tensor(0.0234)
epoch:  150	argmax time taken,  0.006744384765625
assembling the matrix time taken:  0.0002651214599609375
solving Ax = b time taken:  0.0074465274810791016
		 time taken minimize linear layer:  0.007741689682006836
cu

...

solving Ax = b time taken:  0.020258426666259766
		 time taken minimize linear layer:  0.020667076110839844
current error:  tensor(0.0092)
epoch:  369	argmax time taken,  0.013406753540039062
assembling the matrix time taken:  0.0002624988555908203
solving Ax = b time taken:  0.02039194107055664
		 time taken minimize linear layer:  0.020684480667114258
current error:  tensor(0.0092)
epoch:  370	argmax time taken,  0.016858577728271484
assembling the matrix time taken:  0.0002529621124267578
solving Ax = b time taken:  0.020409107208251953
		 time taken minimize linear layer:  0.02069234848022461
current error:  tensor(0.0092)
epoch:  371	argmax time taken,  0.013418912887573242
assembling the matrix time taken:  0.000255584716796875
solving Ax = b time taken:  0.02042412757873535
		 time taken minimize linear layer:  0.020710468292236328
current error:  tensor(0.0091)
epoch:  372	argmax time taken,  0.016835689544677734
assembling the matrix time taken:  0.0002532005310058594
solving 

solving Ax = b time taken:  0.02262091636657715
		 time taken minimize linear layer:  0.023025989532470703
current error:  tensor(0.0085)
epoch:  402	argmax time taken,  0.015608549118041992
assembling the matrix time taken:  0.0002529621124267578
solving Ax = b time taken:  0.02390909194946289
		 time taken minimize linear layer:  0.02419257164001465
current error:  tensor(0.0085)
epoch:  403	argmax time taken,  0.013895273208618164
assembling the matrix time taken:  0.00025534629821777344
solving Ax = b time taken:  0.022704601287841797
		 time taken minimize linear layer:  0.022991418838500977
current error:  tensor(0.0084)
epoch:  404	argmax time taken,  0.015668392181396484
assembling the matrix time taken:  0.0002532005310058594
solving Ax = b time taken:  0.02387261390686035
		 time taken minimize linear layer:  0.024156570434570312
current error:  tensor(0.0084)
epoch:  405	argmax time taken,  0.013913393020629883
assembling the matrix time taken:  0.00025773048400878906
solvin

solving Ax = b time taken:  0.024007320404052734
		 time taken minimize linear layer:  0.024418354034423828
current error:  tensor(0.0078)
epoch:  438	argmax time taken,  0.01611614227294922
assembling the matrix time taken:  0.0002541542053222656
solving Ax = b time taken:  0.024944782257080078
		 time taken minimize linear layer:  0.02523016929626465
current error:  tensor(0.0078)
epoch:  439	argmax time taken,  0.014362573623657227
assembling the matrix time taken:  0.0002541542053222656
solving Ax = b time taken:  0.02411031723022461
		 time taken minimize linear layer:  0.024395227432250977
current error:  tensor(0.0078)
epoch:  440	argmax time taken,  0.016148805618286133
assembling the matrix time taken:  0.0002541542053222656
solving Ax = b time taken:  0.024921178817749023
		 time taken minimize linear layer:  0.025206565856933594
current error:  tensor(0.0078)
epoch:  441	argmax time taken,  0.01435708999633789
assembling the matrix time taken:  0.0003209114074707031
solving 

epoch:  473	argmax time taken,  0.016895055770874023
assembling the matrix time taken:  0.00025963783264160156
solving Ax = b time taken:  0.028345823287963867
		 time taken minimize linear layer:  0.028636455535888672
current error:  tensor(0.0072)
epoch:  474	argmax time taken,  0.014875650405883789
assembling the matrix time taken:  0.0002741813659667969
solving Ax = b time taken:  0.028536319732666016
		 time taken minimize linear layer:  0.02884197235107422
current error:  tensor(0.0072)
epoch:  475	argmax time taken,  0.016951799392700195
assembling the matrix time taken:  0.0002532005310058594
solving Ax = b time taken:  0.028467655181884766
		 time taken minimize linear layer:  0.02875232696533203
current error:  tensor(0.0072)
epoch:  476	argmax time taken,  0.014928579330444336
assembling the matrix time taken:  0.0002620220184326172
solving Ax = b time taken:  0.028558015823364258
		 time taken minimize linear layer:  0.028974533081054688
current error:  tensor(0.0072)
epoch

epoch:  507	argmax time taken,  0.01734137535095215
assembling the matrix time taken:  0.00025653839111328125
solving Ax = b time taken:  0.029596805572509766
		 time taken minimize linear layer:  0.02988457679748535
current error:  tensor(0.0068)
epoch:  508	argmax time taken,  0.01533198356628418
assembling the matrix time taken:  0.00025343894958496094
solving Ax = b time taken:  0.029537439346313477
		 time taken minimize linear layer:  0.029822349548339844
current error:  tensor(0.0068)
epoch:  509	argmax time taken,  0.017416715621948242
assembling the matrix time taken:  0.00025200843811035156
solving Ax = b time taken:  0.029595136642456055
		 time taken minimize linear layer:  0.029877424240112305
current error:  tensor(0.0067)
epoch:  510	argmax time taken,  0.015444278717041016
assembling the matrix time taken:  0.0002524852752685547
solving Ax = b time taken:  0.029601097106933594
		 time taken minimize linear layer:  0.030005693435668945
current error:  tensor(0.0067)
epoc